# T015 · Binding site detection

Authors:

- Andrea Volkamer, 2020, [Volkamer lab, Charité](https://volkamerlab.org/)
- Dominique Sydow, 2020, [Volkamer lab, Charité](https://volkamerlab.org/)

## Aim of this talktorial

Short summary of this talktorial's content.

### Contents in *Theory*

*These points should refer to the headlines of your theory section below.*

* ChEMBL database
* Compound activity measures

### Contents in *Practical*

*These points should refer to the headlines of your practical section below.*

* Connect to ChEMBL database
* Load and draw molecules

### References

* Paper 
* Tutorial links
* Other useful resources

*We suggest the following citation style:*
* Keyword describing resource: <i>Journal</i> (year), <b>volume</b>, pages (link to resource) 

*Example:*
* ChEMBL web services: <i>Nucleic Acids Res.</i> (2015), <b>43</b>, 612-620 (https://academic.oup.com/nar/article/43/W1/W612/2467881) 

## Theory

### ChEMBL database

Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet.

**Note**: Include images like this (place images in `images/`):

![ChEMBL web service schema](images/chembl_webservices_schema_diagram.jpg)

*Figure 1:* 
Describe figure and add reference.
Figure and description taken from: [<i>Nucleic Acids Res.</i> (2015), <b>43</b>, 612-620](https://academic.oup.com/nar/article/43/W1/W612/2467881).

### Compound activity measures

Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet.

Please use LaTeX to format formulas: $pIC_{50} = -log_{10}(IC_{50}*10^{-9})= 9-log_{10}(IC_{50}) $

If you place links, please link descriptive words.
> **Yes**:  This figure is licensed under [CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/).

> **No**: This figure is licensed under CC BY-SA 3.0 ([here](https://creativecommons.org/licenses/by-sa/3.0/)).

## Practical

Short summay of what will be done in this practical section.

Please add all your imports on top of this section.

In [3]:
from pathlib import Path

import pandas as pd
from opencadd.databases.klifs import setup_remote

pd.set_option('display.max_columns', 50)

Add globals to this talktorial's path (`HERE`) and its data folder (`DATA`).

In [2]:
HERE = Path(_dh[-1])
DATA = HERE / "data"

### Define kinase structure of interest

In [5]:
pdb_id = "3w32"

### Get KLIFS pocket

We are using the `opencadd.databases.klifs` module to extract the binding site residues (PDB residue numbering) as defined in the KLIFS database.

Please refer to __Talktorial T012__ for detailed information about KLIFS and the `opencadd.databases.klifs` module usage.

In [4]:
session = setup_remote()

INFO:opencadd.databases.klifs.api:Set up remote session...
INFO:opencadd.databases.klifs.api:Remote session is ready!


In [8]:
# Get first structure KLIFS ID associated with PDB ID
structures = session.structures.by_structure_pdb_id(pdb_id)
structure_klifs_id = structures["structure.klifs_id"].iloc[0]
structure_klifs_id
# Get the structure's pocket
pocket = session.pockets.by_structure_klifs_id(structure_klifs_id)
pocket

,residue.klifs_id,residue.id,residue.klifs_region_id,residue.klifs_region,residue.klifs_color
0,1,716,I.1,I,khaki
1,2,717,I.2,I,khaki
2,3,718,I.3,I,khaki
3,4,719,g.l.4,g.l,green
4,5,720,g.l.5,g.l,green
...,...,...,...,...,...
80,81,855,xDFG.81,xDFG,cornflowerblue
81,82,856,xDFG.82,xDFG,cornflowerblue
82,83,857,xDFG.83,xDFG,cornflowerblue
83,84,858,a.l.84,a.l,cornflowerblue


## Discussion

Wrap up the talktorial's content here and discuss pros/cons and open questions/challenges.

## Quiz

Ask three questions that the user should be able to answer after doing this talktorial. Choose important take-aways from this talktorial for your questions.

1. Question
2. Question
3. Question